In [1]:
##########################
# 配置运行环境
##########################

%matplotlib inline

import numpy as np
import pandas as pd
from IPython.display import Math, Latex
from matplotlib import pyplot
import seaborn as sns

# matplotlib 对中文的支持
from matplotlib import font_manager
cn_font = font_manager.FontProperties(fname='msyh.ttf', size=16)  # 网上支持中文

from matplotlib import rcParams
# rcParams['font.family'] = 'Microsoft YaHei'  # 本地支持中文

# 保存为 pdf 格式
rcParams['pdf.fonttype'] = 42
rcParams['figure.figsize'] = (8,5)

# Look pretty...
from matplotlib import style
style.use('ggplot')

# 设置 numpy 的输出精度, 并且阻止使用科学记数法
# formatter 参数允许为每个类型指定一个格式化函数,
# linewidth 控制输出宽度
np.set_printoptions(precision=6, suppress=True)

\section{21 点简介}

21 点是赌场中流行的一种扑克牌游戏.~玩家不断决定是要庄家再发给他一张牌 (称为 ``拿牌'') 还是不要 (称为 ``停牌'').~玩家若停牌，庄家就给自已发一张牌.~最后，玩家和庄家比较各自手中牌的总和 (有头像的牌算 10，$A$ 算 1 或 11 看情况而定).~在总和都没有超出 21 时，谁离 21 更近谁赢.~比如，如果你的牌是 K，6，3，A 各一张，总和就是 20; 如果庄家的牌是 Q,5,4 各一张，他的总和就是 19.~这一局你赢.

21 点的规则随赌场的不同而略有变化，但通常都包含以下几条：
\begin{itemize}
\item 庄家发出的头一张牌是明牌，即便之前你还未决定是拿牌还是停牌.

\item 如果你和庄家各自手上的牌的总和都没有超出 21，谁的总和更大，谁赢.

\item 如果你和庄家牌的总和相等，这一轮就算平，你下的赌注退还给你.

\item 如果你发到的头两张牌的总和恰好是 21 (比如 1 张 A 和 1 张 Q) 那就以你下的赌注的 1.5 倍赔给你，而不管庄家手上的牌如何.

\item 如果愿意，玩家还可作出有利于自己的各种选择.~比如，分牌 (若头两张牌一样，可以把它们分开，玩两手不同的牌)，双倍下注 (拿到头两张牌以后，可以把赌注加倍，然后只能再拿 1 张牌)，保险 (如果庄家发出的第 1 张明牌是 A，你可以再加注，赌下一张牌是 10 或有头像的牌)，放弃 (立即停牌，只损失所下赌注的一半).

\item 庄家别无选择，只能一张张牌发下去，直到自己手上的牌的总和等于 17 或更大，这时他就必须停牌了.~(在有些赌场里，如果庄家自己手上的牌的总和是 ``软17''，如牌为 A-6 或 A-4-2，他要再给自已发 1 张牌).
\end{itemize}

乍一看，这些规则似乎非常公正.~谁的牌的总和更大 (没有超出 21)，谁就赢，对于平局的处理也无可挑剔.~此外，玩家还享有某些特殊的权益，比如总和为 21 时会得到额外的赔付，玩牌时可作各种别的选择；而庄家则必须按预定好的程序别无选择.~总的说来，规则似乎是对玩家而不是庄家有利.~然而，各地的赌场却能从 21 点上赚到丰厚的利润.~这是怎么回事呢？

事实上，奥妙在于如果玩家的牌总和超出了 21 就算他输，而不管庄家的牌如何.~换句话说，如果双方各自的牌总和都超出了21 (或若游戏再玩下去将要超出 21)，就算庄家赢.~这是唯一对赌场有利的规则，但有它就够了，足以让钱财滚滚而来.

虽然 21 点给玩家提供了很多选择，但核心是何时停牌何时拿牌.~显然，如果你的牌总和是 11 或更少，你得拿牌.~如果总和是 20 或 21，你得停牌.~但如果你的牌总和是15，又该怎么办呢？在这种情况下，如果拿牌，可能会发到一张 5 或 6，那么牌更好了.~但你也可能会发到 7 至 K 之间的某一张牌，结果就爆牌了.~现在该怎么做呢？

基本原则同其他牌相同：所有没有看到的牌随后都会以相同的概率出现.~不过，有一点不同.~赌场中玩 21 点时，一般会把很多副牌 (6 副或更多副) 混在一起洗，并且常常是洗了又洗.~因此，前面发出了哪几张牌，对于接下来会发出怎样的一张牌，几乎不会有什么影响.~另外，大多数赌场都明令禁止玩家清点到目前为止己现身的牌；如果有谁被发现这样做，赌场就会对他下逐客令.~(借助一些记忆窍门，玩家也许能对剩下的牌中大张与小张的比例有所知晓，一些有经验的玩家也确实由此取得过成功，但如果用更多副牌并且洗的次数也越来越多，那么这种做法的效果肯定会越来越
差.~) 所以我们假定玩21 点时，从A到K的每一张牌都会以相同的概率在随后出现，而不管前面已经发出了什么牌.

假设你在赌场中玩 21 点 (用了很多副牌)，并且已经发到一张 J 和一张 8 (和为 18).~下一张发给你的牌可能是 A 到 K 的 13 张牌中的任何一张，它们里面有 1O 张 (4 及以上的牌) 会让你爆牌.~所以，你超出 21 的概率是 $10/13$，即 $77\%$.~这也太大了，此刻你最好停牌.

如果在和为 18 时停牌，下面会出现什么情况呢？这时庄家就必须拿牌，直到他的和为 17 或更大，而不管你的牌如何.~长远来看，庄家手上的牌之和的各种概率 (用了很多副牌)，如表4.2所示.
\begin{table}[H]\centering
\begin{tabular}{cc}
庄家牌最后的和 & 概率 \\
17 & 15.47\% \\
18 & 14.76\% \\
19 & 14.00\% \\
20 & 18.50\% \\
21 & 9.55\% \\
爆牌 & 27.73\%
\end{tabular}
\end{table}

由上表可知，当玩家牌的和为 18 时，如果庄家牌的和为 17 或超出 21，那玩家赢得这一局的概率约是 43\%.~如果庄家的和也是 18，那
双方平的概率约是 15\%.~如果庄家的和是 19, 20 或 21，那玩家输牌的概率约是 42\%.~所以，和为 18 时停牌，这一局牌双方基本上不分胜负.~实际上，玩家还略微占先呢.~这当然比起拿牌使得结果有 77\% 的
概率会立即超过 21 点要好.

除了作上述考虑外，还有一张牌你应特别加以注意.~那就是庄家发给自己的第一张牌 (明牌)，这张牌对庄家最后可能得到的和有很大的影响.~根据这第一张牌，长远来看庄家手上的牌的和及其相应的概率 (在很多副牌的情况下) 如表4.3所示.

\begin{table}
\begin{tabular}{ccccccc}
笫一张牌 & 17 & 18 & 19 & 20 & 21 & 爆牌 \\
A & 13.41\% & 13.41\% & 13.41\% & 13.41\% & 36.48\% & 9.89\% \\
2 & 14.64\% & 14.03\% & 13.37\% & 12.66\% & 11.90\% & 33.41\% \\
3 & 14.16\% & 13.59\% & 12.98\% & 12.32\% & 11.61\% & 35.33\% \\
4 & 13.68\% & 13.15\% & 12.60\% & 11.97\% & 11.31\% & 37.32\% \\
5 & 13.19\% & 12.70\% & 12.17\% & 11.61\% & 10.99\% & 39.33\% \\
6 & 12.48\% & 12.03\% & 11.54\% & 11.01\% & 10.44\% & 42.50\% \\
7 & 38.50\% & 9.51\% & 9.05\% & 8.56\% & 8.03\% & 26.34\% \\
8 & 14.31\% & 37.39\% & 8.39\% & 7.94\% & 7.45\% & 24.52\% \\
9 & 13.28\% & 13.28\% & 36.36\% & 7.36\% & 6.91\% & 22.82\% \\
10 及以上 & 12.31\% & 12.31\% & 12.31\% & 35.39\% & 6.39\% & 21.28\%
\end{tabular}
\end{table}

上表表明，庄家的和是某个数的概率与他的第一张牌有很大的关系.~第一张牌是 A，那就很灵活 (因为它可算成 1 也可算成 11)，最终爆牌的概率也因此能降到10\% 以下，而且和为 21 的概率很大 (只要下一张牌是 10 或有头像的牌就行).~然而，第一张牌若是 6，庄家最终爆牌的概率就非常大，达到 42.5\% (如果下一张牌是 10 或有头像的牌，和就是 16，再往后就可能会爆牌了).~相比之下，第一张牌是 7，和为 17 的概率很大；第 1 张牌是 9，和为 19 的概率很大，以此类推，因为下一张牌很可能是 10 或有头像的牌.

知道这些又有什么用呢？假设你己拿到 1 张 J 和 1 张 3 (和为13)，庄家的第一张牌是5.~你该拿牌还是停牌？如果拿牌且拿到的牌又是 9, 10, J, Q 或 K 之一，那你就会爆牌，就输了 --- 这一概率是 5/13，即 38.5\%.~当然，你的和也可能不会超出 21，但不一定就能赢得这一局.~总的来说，如果你再拿牌，赢的概率与所拿的下一张牌有关，如表4.4所示.

\begin{table}
\begin{tabular}{ccccc}
拿到的下一张牌 & 拿到此牌的概率 & 总和 & 玩家嬴的概率 & 双方打平的概率 \\
A & 1/13 & 14 & 39.33\% & 0\% \\
2 & 1/13 & 15 & 39.33\% & 0\% \\
3 & 1/13 & 16 & 39.33\% & 0\% \\
4 & 1/13 & 17 & 39.33\% & 13.19\% \\
5 & 1/13 & 18 & 52.52\% & 12.70\% \\
6 & 1/13 & 19 & 65.22\% & 12.17\% \\
7 & 1/13 & 20 & 77.39\% & 11.61\% \\
8 & 1/13 & 21 & 89.01\% & 10.99\% \\
9 & 1/13 & 22 & 0\% & 0\% \\
10及以上 & 4/13 & 23 & 0\% & 0\% \\
平均 & & & 33.96\% & 4.67\%
\end{tabular}
\end{table}

把表4.4的第4列、第5列中的各种概率 (它们与发给你的下一张牌有关) 分别平均一下，即可算出：如果你再拿牌一次，赢的概率是 33.96\%，打平的概率是4.67\%.

别忘了，庄家自己的第一张牌是 5，这并不是一张好牌.~从表4.3可以看出，庄家最后爆牌的概率是39.33\%.~所以，如果你停牌，虽然和只有微不足道的 13，但赢的机会仍不算低 (39.33\%).~停牌，赢的概率为39.33\%; 再拿牌一次，赢的概率为33.96\% (即使加上打平，也只有 38.63\%).~因此，如果你发到的牌是 1 张 J 和 1 张 3，庄家自己的第一张牌是 5，总的来看，你应选择停牌，这比选择拿牌要好.

认真的 21 点玩家们会详细地作过此类计算，包括应用计算机模拟以求出各种不同的概率.~他们已经提出了一种 ``基本策略''，目的是在各种不同情况下使赢的概率达到最大.~这一策略有详尽的规则，说明何时该分牌，何时该双倍下注，何时该拿牌，何时该停牌.~(应用这一基本策略去玩 21 点，可以把赌场一方的占先优势降低到 0.5\% 刚过，但即便如此，从长远来看玩家仍要输钱).~这一基本策略中有一条规则是说，如果玩家当前的和在 13 与 16 之间且没有 A 算作 11，并且庄家的第一张牌在 2 与 6 之间，那玩家应该停牌.~这与我们所举的牌为 J 和 3 的那个例子一样：如果你选择拿牌，和可能会更好，但也可能会超出21.~总的来看，在这种清况下最好是停牌，寄希望于庄家爆牌.

在玩与运气有关的游戏时，对随机性的理解能增加赢的概率.~不过，真要成为赢家，最终需要的是耐心.~大数定律说了，长远来看，赢的概率最大的人会赢得最多.~这并不是说每次玩你都会赢，而是说如果这一游戏你玩了又玩，就会比别人赢得更多.~一旦搞清了怎样增加赢的概率，你也许就要玩许多许多次，以便最终走上赢的道路.~ (类似的情况也适用于股票市场成功并不在于你所有的股票都得一直往上涨，而是在于平均来看它们是往上涨.)

不幸的是，有时成功不能复制.~这种情况下，你能做的就是让赢的概率尽量地大，并且把握住机会.~要想在玩与运气有关的游戏时胜出，得具备三个条件.~首先，得仔细研究游戏本身，看能否找到一种策略，能让你平均来看会赢.~其次，得一再地重复应用那种策略.~第三，得耐心地等着大数定律最终把你引向成功.